In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import odr

# 1. Boostrap basics

In [ ]:
np.random.seed(30)
normal_data = np.random.normal(4, 1.5, 50)

To the best of the GSIs' knowledge, there is no perfect package for boostrap validation in `Python`. Therefore, the function below is a simple manual implementation of the bootstrap. Note that there is nothing tricky in the code below and you are encouraged to try the implementation on your own.

In [ ]:
def bootstrap(data, metrics,sample =500, random_state=10):
    output_array=np.zeros(sample)
    # output_array[:]=np.nan
    for i in range(sample):
        bs_data = np.random.choice(data, len(data), replace=True)
        output_array[i] = metrics(bs_data)
    return output_array

In [ ]:
boot_out = bootstrap(normal_data, np.mean)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,5))
axs[0].set_xlabel('Data observation', fontsize=16)
axs[1].set_xlabel('Mean estimate', fontsize=16)
axs[0].set_ylabel('Count', fontsize=16)
axs[0].hist(normal_data,bins=30,edgecolor='red', linewidth=2,color = "grey")
axs[1].hist(boot_out,bins=30,edgecolor='blue', linewidth=2,color = "grey")

Using the bootstrap output, you can compute the stiatistics (e.g., mean, standard deviation, quantiles) of the boostrap estimates.

In [ ]:
print("mean of the estimate %.2f" % np.mean(boot_out))
print("sd of the estimate: %.2f " % np.std(boot_out))
print("0.025-quantile of the estimate: %.2f " % np.quantile(boot_out, 0.025))
print("0.975-quantile of the estimate: %.2f " % np.quantile(boot_out, 0.975))

## 2. Bootstrap for Model Validation 

### 2.1 Preliminaries

#### 2.1.1 From Lab 7

In [ ]:
ctr = pd.read_csv("CTR.csv")

from sklearn.model_selection import train_test_split

y = ctr['CTR']
X = pd.get_dummies(ctr.drop(['CTR'], axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=88)
X_train.shape, X_test.shape

In [ ]:
def OSR2(model, X_test, y_test, y_train):
    
    y_pred = model.predict(X_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

Note that we take the random forest model as an example; However, bootstrap for valiation can be applied to any trained model.

Below, we train the random forest model, on the training data and with the best parameters found through cross-validation in Lab 7.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_features=11, min_samples_leaf=5, 
                           n_estimators = 500, random_state=10, verbose=1)
# Note: you can change the verbose parameter to control how much training progress is printed.
rf.fit(X_train, y_train)

In [ ]:
rf.verbose = False
print('OSR2:', round(OSR2(rf, X_test, y_test, y_train), 5))

#### 2.1.2 Define metrics functions

In [ ]:
def OS_R_squared(predictions, y_test,y_train):
    SSE = np.sum((y_test-predictions)**2)
    SST = np.sum((y_test-np.mean(y_train))**2)
    r2 = 1-SSE/SST
    return r2

def mean_squared_error(predictions, y_test,y_train):
    MSE = np.mean((y_test-predictions)**2)
    return MSE

def mean_absolute_error(predictions, y_test,y_train):
    MAE = np.mean(np.abs(y_test-predictions))
    return MAE

#### 2.1.3 Sanity check

In [ ]:
y_pred = rf.predict(X_test)
print("OSR2: %s" % OS_R_squared(y_pred,y_test,y_train))
print("MSE: %s" % mean_squared_error(y_pred,y_test,y_train))
print("MAE: %s" % mean_absolute_error(y_pred,y_test,y_train))

In [ ]:
print("OSR2: %s" % OSR2(rf, X_test, y_test,y_train))
print("MSE: %s" % np.mean((y_pred-y_test)**2))
print("MAE: %s" % np.mean(np.abs(y_pred-y_test)))

### 2.2 Boostrapping

#### 2.2.1 Manually code the bootstrap method

Again, below is a manual implementation of bootstrap for model valiation.

In [ ]:
import time

def bootstrap_validation(test_data, test_label, train_label, model, metrics_list, sample=500, random_state=66):
    tic = time.time()
    n_sample = sample
    n_metrics = len(metrics_list)
    output_array=np.zeros([n_sample, n_metrics])
    output_array[:]=np.nan
    print(output_array.shape)
    for bs_iter in range(n_sample):
        bs_index = np.random.choice(test_data.index, len(test_data.index), replace=True)
        bs_data = test_data.loc[bs_index]
        bs_label = test_label.loc[bs_index]
        bs_predicted = model.predict(bs_data)
        for metrics_iter in range(n_metrics):
            metrics = metrics_list[metrics_iter]
            output_array[bs_iter, metrics_iter]=metrics(bs_predicted,bs_label,train_label)
#         if bs_iter % 100 == 0:
#             print(bs_iter, time.time()-tic)
    output_df = pd.DataFrame(output_array)
    return output_df

#### 2.2.2 Do bootstrap

Note that here we use B = 5,000, instead of 10,000 as shown in the lecture. Later, you will observe in the histogram that with this resampling size, the estimates of the metrics have already displayed the nice normal curve shape.

The following code takes about 10 minutes to run. You might want to reduce the value of B for a trial run. However, B should not be too small; otherwise, the estimates may not converge to the standard normal distribution well.

In [ ]:
bs_output = bootstrap_validation(X_test,y_test,y_train,rf,
                                 metrics_list=[OS_R_squared, mean_squared_error,mean_absolute_error],
                                 sample = 5000)

### 2.3 Visualization

#### 2.3.1 Basic plot and centered plot

In [ ]:
test_OSR2 = OS_R_squared(y_pred,y_test,y_train)

fig, axs = plt.subplots(ncols=2, figsize=(12,5))
axs[0].set_xlabel('Bootstrap OSR2 Estimate', fontsize=16)
axs[1].set_xlabel('Boot OSR2 - Test Set OSR2', fontsize=16)
axs[0].set_ylabel('Count', fontsize=16)
axs[0].hist(bs_output.iloc[:,0], bins=20,edgecolor='green', linewidth=2,color = "grey")
axs[0].set_xlim([0.4,0.7])
axs[1].hist(bs_output.iloc[:,0]-test_OSR2, bins=20,edgecolor='green', linewidth=2,color = "grey")
axs[1].set_xlim([-0.15,0.15])

#### 2.3.2 manual CI + plot

In [ ]:
# The 95% confidence interval
CI= np.quantile(bs_output.iloc[:,0]-test_OSR2,np.array([0.025,0.975]))
print("The 95-percent confidence interval of OSR2 is %s" % CI)

In [ ]:
fig, axs = plt.subplots(ncols=1, figsize=(8,6))
axs.set_xlabel('Boot OSR2 - Test Set OSR2', fontsize=16)
axs.set_ylabel('Count', fontsize=16)
axs.hist(bs_output.iloc[:,0]-test_OSR2, bins=20,edgecolor='green', linewidth=2,color = "grey")
axs.set_xlim([-0.15,0.15])
axs.vlines(x=CI[0], ymin = 0, ymax =800, color = "black")
axs.vlines(x=CI[1], ymin = 0, ymax =800, color = "black")

### 2.4 Exercise

Try to re-do this exercise with another models that we did in Lab 7, e.g., linear regression, decision tree, and bagging

## The idea behind boosting

We have seen with Bagging and Random Forest how to use the power of model ensemble to enhance model performance. Another interesting idea is adaptively combined simple models. How?
1. Start simple, use linear regression for a regression task --> h_1(X). 
2. Take the training set response and subtract the predicted value R_1(X) = y - h_1(X).
3. Fit another easy model, i.e a square model to the residual R_1 --> h_2(X)
4. Repeat until a stopping criterion is met

This is powerful because is numerically better than running the regression on [1, X, X^2] and this is because, instead of the Vandermonde matrix, we use a basis change to obtain orthogonal basis fro the same space (--> orthogonal means no problem with multicollinearity!!!!) 

In class, you have seen a nice example with a 4th degree polynomial, we are gonna reproduce it and show another one.

In [ ]:
x = np.linspace(0.0, 5.0)
y = np.sin(x)
poly_model = odr.polynomial(3)  # using third order polynomial model
data = odr.Data(x, y)
odr_obj = odr.ODR(data, poly_model)
output = odr_obj.run()  # running ODR fitting
poly = np.poly1d(output.beta[::-1])
poly_y = poly(x)
plt.plot(x, y, label="input data")
plt.plot(x, poly_y, label="polynomial ODR")
plt.legend()
plt.show()

In [ ]:
x = np.linspace(0.0, 5.0)
y = np.sin(x)
poly_model = odr.polynomial(3)  # using third order polynomial model
data = odr.Data(x, y)
odr_obj = odr.ODR(data, poly_model)
output = odr_obj.run()  # running ODR fitting
poly = np.poly1d(output.beta[::-1])
poly_y = poly(x)
plt.plot(x, y, label="input data")
plt.plot(x, poly_y, label="polynomial ODR")
plt.legend()
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
x = np.linspace(-2, 4, 100).reshape(-1, 1)
state = np.random.seed(10)
eps = np.random.normal(0, 5, size=(100, 1))
true_y = x**4 - 3*x**3 - 2*x**2 +5*x + 10
y = x**4 - 3*x**3 - 2*x**2 +5*x + 10 + eps
plt.plot(x, true_y, label="input data")
plt.scatter(x, y, label="noisy data", color='orange')
plt.legend()
plt.show()

In [ ]:
model1 = LinearRegression().fit(x, y)
intercept_1 = model1.intercept_
coefficients_1 = model1.coef_
prediction_1 = model1.predict(x)
residuals_1 = y - prediction_1
print('Intercept:', intercept_1)
print('Coefficients:', coefficients_1)
plt.plot(x, true_y, label="input data")
plt.plot(x, intercept_1 + coefficients_1*x, label="linear regression")
plt.scatter(x, residuals_1, label="residual after the first regression", color = 'g')
plt.legend()
plt.show()

In [ ]:
X = np.hstack((x, x**2))
model2 = LinearRegression().fit(X, residuals_1)
intercept_2 = model2.intercept_
coefficients_2 = np.array(model2.coef_.reshape(-1, 1))
prediction_2 = model2.predict(X)
residuals_2 = residuals_1 - prediction_2
print('Intercept:', intercept_2)
print('Coefficients:', coefficients_2)
plt.plot(x, true_y, label="input data")
plt.plot(x, residuals_1, label="residual for the first regression")
plt.plot(x, intercept_2 + coefficients_2[0]*X[:,0] + coefficients_2[1]*X[:,1], label="quadratic regression on r1")
plt.scatter(x, residuals_2, label="residual after the second regression", color='r')
plt.legend()
plt.show()

In [ ]:
X = np.hstack((x, x**2, x**3))
model3 = LinearRegression().fit(X, residuals_2)
intercept_3 = model3.intercept_
coefficients_3 = np.array(model3.coef_.reshape(-1, 1))
prediction_3 = model3.predict(X)
residuals_3 = residuals_2 - prediction_3
print('Intercept:', intercept_3)
print('Coefficients:', coefficients_3)
plt.plot(x, true_y, label="input data")
plt.plot(x, intercept_3 + coefficients_3[0]*X[:,0] + coefficients_3[1]*X[:,1] + coefficients_3[2]*X[:,2], label="Degree 3 regression on r2")
plt.scatter(x, residuals_3, label="residual after the third regression", color='g')
plt.legend()
plt.show()

In [ ]:
X = np.hstack((x, x**2, x**3, x**4))
model4 = LinearRegression().fit(X, residuals_3)
intercept_4 = model4.intercept_
coefficients_4 = np.array(model4.coef_.reshape(-1, 1))
prediction_4 = model4.predict(X)
residuals_4 = residuals_3 - prediction_4
print('Intercept:', intercept_4)
print('Coefficients:', coefficients_4)
plt.plot(x, true_y, label="input data")
plt.plot(x, intercept_4 + coefficients_4[0]*X[:,0] + coefficients_4[1]*X[:,1] + coefficients_4[2]*X[:,2]
         + coefficients_4[3]*X[:,3], label="Degree 4 regression on r3")
plt.scatter(x, residuals_4, label="residual after fourth regression", color='g')
plt.legend()
plt.show()

In [ ]:
final_prediction = prediction_1+prediction_2+prediction_3+prediction_4

In [ ]:
plt.plot(x, true_y, label="input data")
plt.scatter(x, final_prediction, label="ensemble prediction", color='g')